## Set up paths and imports

In [ ]:
import os

import torch
from torchvision import transforms
import matplotlib.pyplot as plt

if not os.path.exists("./notebooks"):
    %cd ..

import src.model
import numpy as np
from src.training import predict
from src.data_processing import load_mean_std
from src.config import DATASET_DIR

## Load Model and Dataset

In [ ]:
from src.dataset import prepare_dataset_loaders

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


name = "OriginalSizeCNN"
model = src.model.OriginalSizeCNN()
model_path = f"./models/{name}.pth"
model.load_state_dict(torch.load(model_path, weights_only=True))

c1_layer = src.model.SingleLayerModel(model.conv1)
c1_layer.device = device
batch_size = 32

mean, std = load_mean_std(f"{DATASET_DIR}/scaling_params.json")
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])
train_loader, val_loader, test_loader = prepare_dataset_loaders(transform, 32)

output = c1_layer(test_loader.dataset[0][0])#]
print(output.shape)

output = output.detach().numpy()#c1_predicts = np.array(c1_predicts)
print(output.shape)



## Visualize feature map

In [ ]:
def plot_feature_map(feature_map, max_grid):
    
    fig, ax = plt.subplots(max_grid, max_grid, figsize=(7,7))
    channel_idx = 0
    
    for i in range(max_grid):
        for j in range(max_grid):
            ax[i][j].imshow(feature_map[channel_idx,:,:])
            ax[i][j].axis('off')
            
            channel_idx += 1
            
    fig.suptitle(f'Feature Map - Displaying {max_grid**2} of {feature_map.shape[0]} Channels')
    fig.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()

In [ ]:
plot_feature_map(output, 4)